In [2]:
from agent.agent_functions import create_program

In [14]:
# program = develop("Create a fastAPI app with a generic Readme.md file and a main.py file with some basic math function endpoints. The readme should have a title, a description, and a table of contents and contain useful examples. The main.py file should have a function called add that takes in two numbers and returns their sum. It should also have a function called subtract that takes in two numbers and returns their difference.")
# program = develop("Develop a simple Python program that calculates the factorial of a number. The program should consist of two files. The first file should be named factorial.py and should contain a function named factorial that takes an integer as input and calculates its factorial. The second file should be named main.py and should import the factorial function from factorial.py. It should take an integer input from the user, call the factorial function with this input, and print the resulting factorial.")
# program = develop("""Create a Python Flask application that includes three files: app.py, views.py, and requirements.txt.

# # In the app.py file, initialize the Flask app and include routes for home, about, and error 404. Make sure to include the necessary imports and to use the Flask render_template function to render the HTML templates for each route.

# # The views.py file should contain the functions that define what happens at each route. The home route should return a message that says 'Welcome to the Home Page!'. The about route should return a message that says 'About Page'. Lastly, the error 404 route should return '404 Page Not Found'.

# # The requirements.txt file should list all the necessary packages to run the Flask application, including Flask itself.

# # Please ensure that the Flask application can be run correctly and that it responds to the specified routes with the appropriate messages.""")
program = create_program("""
I want a basic RESTful API using Python Flask and SQLAlchemy that provides basic CRUD operations on a single Tasks resource.

Each Task will have the following attributes:

id: A unique identifier for the task. This should be an auto-incrementing integer.
title: A short descriptive title for the task. This should be a string of maximum 100 characters.
description: A longer text giving more details about the task. This should be a string of maximum 1000 characters.
done: A status to indicate whether the task has been completed or not. This should be a boolean.
The API should have the following endpoints:

GET /tasks : Returns a list of all tasks in the database.
POST /tasks : Creates a new task. The request body should be a JSON object containing 'title', 'description' and 'done'.
GET /tasks/<task_id> : Returns the details of a single task with the provided task id.
PUT /tasks/<task_id> : Updates the details of a specific task. The request body should be a JSON object containing the details to be updated.
DELETE /tasks/<task_id> : Deletes a specific task from the database.
The API should be connected to a SQLite database using SQLAlchemy and all data should be persisted in this database.

In addition to the Python scripts for the Flask application and the SQLAlchemy models, please also provide a requirements.txt file that lists all necessary Python packages to run the application.

The root directory should have the following structure:

/app
init.py
models.py
routes.py
run.py
requirements.txt
The run.py script should be the entry point of the application and should start the Flask development server. The /app/__init__.py script should initialize the Flask app and the SQLAlchemy database instance. The /app/models.py script should define the Task SQLAlchemy model. The /app/routes.py script should define all the routes of the Flask application.""")

In [16]:
program = create_program("""
I'd like to create a machine learning microservice using Flask, scikit-learn and Docker. The service will be a sentiment analysis API that will predict sentiment (Positive, Negative, Neutral) based on text data input. Here are the key components that I want:

requirements.txt
This file should include all the necessary packages such as Flask, scikit-learn, pandas, numpy, and joblib.

Dockerfile
A Dockerfile to build a Docker image for the application. It should use a Python base image, copy the application files to the image, install the requirements, and start the application.

app/init.py
This file should create the Flask app.

app/model.py
This file should include a class SentimentModel to load the machine learning model using joblib. The model file is named model.joblib.

app/routes.py
This file should define the API endpoint /predict for POST requests. The POST request will include JSON data in the format { "text": "input text here" }. The endpoint should return the predicted sentiment and the probability in JSON format.

app/utils.py
This file should include utility functions for data preprocessing, which can be used in the prediction endpoint.

tests/test_app.py
This file should contain tests for the application using the pytest framework.

main.py
This file should run the application.

model_training.ipynb
A Jupyter notebook file for training the sentiment analysis model on a dataset. The dataset can be a CSV file with two columns: "text" and "sentiment". The model should be a pipeline that includes data preprocessing steps and a machine learning model (e.g., logistic regression, Naive Bayes, etc.). Save the trained model as model.joblib.
""")

Program(files=[File(name='requirements.txt', contents='Flask==1.1.2\nscikit-learn==0.23.2\npandas==1.1.3\nnumpy==1.19.2\njoblib==0.17.0\npytest==6.1.1'), File(name='Dockerfile', contents='FROM python:3.8\nWORKDIR /app\nCOPY . /app\nRUN pip install -r requirements.txt\nEXPOSE 5000\nCMD ["python", "main.py"]'), File(name='app/__init__.py', contents='from flask import Flask\n\napp = Flask(__name__)\n\nfrom app import routes'), File(name='app/model.py', contents="from joblib import load\n\nclass SentimentModel:\n    def __init__(self):\n        self.model = load('model.joblib')\n    def predict(self, text):\n        return self.model.predict([text])"), File(name='app/routes.py', contents="from flask import request, jsonify\nfrom app import app\nfrom app.model import SentimentModel\n\n@app.route('/predict', methods=['POST'])\ndef predict():\n    data = request.get_json()\n    text = data['text']\n    model = SentimentModel()\n    prediction = model.predict(text)\n    return jsonify({'predic

TypeError: 'Program' object is not subscriptable